First load libraries

In [1]:
from cobra.io import load_json_model, read_sbml_model, load_matlab_model
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import pfba

from dingo.utils import correlated_reactions
from dingo.utils import cluster_corr_reactions
from dingo.utils import graph_corr_matrix
from dingo.illustrations import plot_corr_matrix
from dingo.illustrations import plot_dendrogram
from dingo.illustrations import plot_histogram
from dingo.illustrations import plot_graph
from dingo.preprocess import PreProcess
from dingo import MetabolicNetwork, PolytopeSampler

import numpy as np
import pickle
from functools import reduce
import copy



/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Load models

In [2]:

#roseburia_dingo_model = MetabolicNetwork.from_mat("/home/touliopoulos/project/erasmus/code/data/Reconstructions_data/Roseburia_intestinalis_L1_82_withMucins.mat")
#roseburia_dingo_model = MetabolicNetwork.from_mat("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Reconstructions_data/Roseburia_intestinalis_L1_82_fixed.mat")
#roseburia_cobra_model = load_matlab_model("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Reconstructions_data/Roseburia_intestinalis_L1_82_fixed.mat")
ec_dingo_model = MetabolicNetwork.from_sbml("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Reconstructions_data/e_coli_core.xml")
ec_cobra_model = read_sbml_model("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Reconstructions_data/e_coli_core.xml")

#reactions = roseburia_dingo_model.reactions
reactions = ec_dingo_model.reactions

#print(roseburia_cobra_model.groups)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-29


Now load sampling/correlation matrix data

In [ ]:
# load samples data
#with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/RI_steady_states_opt_1.pckl", "rb") as steady_states:
#with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/EC_steady_states_opt_1.pckl", "rb") as steady_states:
with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/EC_steady_states_opt_100.pckl", "rb") as steady_states:
    steady_states = pickle.load(steady_states)


#np.savetxt("/home/touliopoulos/project/erasmus/erasmus_2025_project/output.tsv", steady_states, delimiter="\t", fmt="%f")


extended_steady_states = []
extended_reactions = []

for index, r in enumerate(steady_states):
    
    if (r < 0).any():
        s = -(r)
        extended_steady_states.append(s)
        extended_reactions.append("_".join([reactions[index], "rev"]))
            
    extended_steady_states.append(r)
    extended_reactions.append(reactions[index])
    
extended_steady_states = np.array(extended_steady_states)
print(steady_states.shape)
print(extended_steady_states.shape)



"""
# list to collect the new rows
extended_steady_states = []
# list to collect the names of the new rows
extended_reactions = []

for index, row in enumerate(steady_states):
    
    #print("\n", row)
    
    # if row has at least one negative value
    if (row < 0).any():
        # find indices of positive (forward reaction) and negative (reverse reaction) values in current row
        negative_mask = row < 0
        positive_mask = row > 0

        
        # create row with only positive values
        forward_reaction = np.where(positive_mask, row, 0)
        print(forward_reaction)
        
        # if forward reaction has at least one non-0 value
        if np.any(forward_reaction) == True:
            extended_steady_states.append(forward_reaction)
            extended_reactions.append("_".join([reactions[index], "for"]))
        
        
        # create row with only negative values
        reverse_reaction = np.where(negative_mask, row, 0)
        #print(reverse_reaction)
        
        # if reverse reaction has at least one non-0 value
        if np.any(reverse_reaction) == True:
            extended_steady_states.append(reverse_reaction)
            extended_reactions.append("_".join([reactions[index], "rev"]))
            

    else:
        extended_steady_states.append(row)
        extended_reactions.append(reactions[index])


# Convert the list back to a NumPy array
extended_steady_states = np.array(extended_steady_states)
"""



"""
rxns = ['PGM', "PGM_rev", 'ENO', 'PGK', 'PGK_rev', 'GAPD', 'PGI', 'TPI', 'FBA', 
          'G6PDH2r', 'PGL', 'GND', 'RPE', 'RPI', 'RPI_rev', 'TKT1', 'TALA', 'TKT2']

indexes = [extended_reactions.index(t) for t in rxns]

print(indexes)

selected_rows = extended_steady_states[indexes]
"""


# load correlation matrix data
#with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/RI_corr_matrix_opt_1.pckl", "rb") as corr_matrix:
#    corr_matrix = pickle.load(corr_matrix)
    

#del steady_states


(95, 4400)
(118, 4400)


'\nrxns = [\'PGM\', "PGM_rev", \'ENO\', \'PGK\', \'PGK_rev\', \'GAPD\', \'PGI\', \'TPI\', \'FBA\', \n          \'G6PDH2r\', \'PGL\', \'GND\', \'RPE\', \'RPI\', \'RPI_rev\', \'TKT1\', \'TALA\', \'TKT2\']\n\nindexes = [extended_reactions.index(t) for t in rxns]\n\nprint(indexes)\n\nselected_rows = extended_steady_states[indexes]\n'

Correlation Matrix and Clustering

In [ ]:

# Remove NAs from corr matrix and find corresponding remaining reactions
#non_na_rows = ~np.all(np.isnan(corr_matrix), axis=1)
#filtered_corr_matrix = corr_matrix[non_na_rows][:, non_na_rows]
#filtered_corr_matrix = np.nan_to_num(filtered_corr_matrix, nan=0)
#filtered_reactions = [reaction for reaction, keep in zip(reactions, non_na_rows) if keep]


#corr_matrix = np.corrcoef(steady_states, rowvar=True)
#corr_matrix[np.isnan(corr_matrix)] = 0


"""
from scipy.stats import spearmanr
corr_matrix, p_values = spearmanr(steady_states, axis=1)
print(corr_matrix.shape)
corr_matrix[np.isnan(corr_matrix)] = 0
corr_matrix[np.abs(corr_matrix) < 0.99] = 0
print(corr_matrix.max())
"""



corr_matrix = correlated_reactions(
                              #selected_rows,
                              extended_steady_states,
                              #steady_states,
                              pearson_cutoff = 0.99,
                              indicator_cutoff = 0,
                              cells = 10,
                              cop_coeff = 0.3,
                              lower_triangle = False)
corr_matrix[np.isnan(corr_matrix)] = 0


plot_corr_matrix(corr_matrix,
                 #reactions= rxns,
                 reactions=extended_reactions, 
                 #reactions=reactions, 
                 format="svg")



dissimilarity_matrix_ward, labels_ward, clusters_ward = cluster_corr_reactions(
                                         corr_matrix,
                                         #reactions=rxns,
                                         reactions=extended_reactions,
                                         #reactions = reactions,
                                         linkage = "ward",
                                         
                                         # 1
                                         t = 2.0,
                                         
                                         # 100
                                         #t = 400,
                                         
                                         correction = False)
print(clusters_ward)

plot_dendrogram(dissimilarity_matrix_ward,
                #reactions=rxns,
                reactions=extended_reactions,
                #reactions = reactions,
                plot_labels = True,
                t = 0.0,
                linkage = "ward")

#del dissimilarity_matrix, labels, clusters



corr_matrix = correlated_reactions(
                              extended_steady_states,
                              pearson_cutoff = 0,
                              indicator_cutoff = 0,
                              lower_triangle = False)

corr_matrix[np.isnan(corr_matrix)] = 0

graphs, layouts = graph_corr_matrix(corr_matrix,
                                    reactions=extended_reactions,
                                    #correction = True,
                                    clusters = [])

plot_graph(graphs[0], layouts[0])






/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning:

invalid value encountered in divide

/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide



[['PGK', 'PGL', 'PGM', 'RPE', 'RPI_rev', 'TALA', 'TKT1', 'TKT2', 'G6PDH2r', 'GND'], ['PGI', 'PGK_rev', 'PGM_rev', 'RPI', 'ENO', 'TPI', 'FBA', 'GAPD'], ['ACKr_rev', 'ACt2r_rev', 'PTAr', 'EX_ac_e'], ['D_LACt2_rev', 'EX_lac__D_e', 'LDH_D_rev'], ['ALCD2x_rev', 'ETOHt2r_rev', 'EX_etoh_e'], ['ACONTa', 'ACONTb', 'CS', 'FUM'], ['CYTBD', 'EX_o2_e_rev', 'O2t'], ['PIt2r', 'BIOMASS_Ecoli_core_w_GAM', 'EX_pi_e_rev'], ['AKGDH', 'SUCOAS_rev', 'ICDHyr'], ['ACKr', 'ACt2r'], ['D_LACt2', 'LDH_D'], ['ALCD2x', 'ETOHt2r'], ['ICL', 'MALS'], ['PPS', 'ADK1'], ['SUCDi', 'FRD7'], ['PFL', 'EX_for_e'], ['SUCCt2_2', 'SUCCt3'], ['PFK', 'ACALD_rev', 'PPC', 'ATPM', 'PPCK', 'ATPS4r', 'PYK', 'THD2', 'EX_fru_e', 'EX_fum_e', 'EX_gln__L_e', 'EX_h_e', 'EX_mal__L_e', 'EX_succ_e', 'FBP', 'FORt2', 'FORt_rev', 'FRUpts2', 'FUMt2_2', 'GLNS', 'GLNabc', 'GLUN', 'MALt2_2', 'MDH', 'ME1', 'ME2', 'NADH16', 'NADTRHD', 'PDH'], ['ACALD', 'FORt'], ['GLUDy_rev'], ['AKGt2r'], ['ACALDt'], ['PYRt2'], ['CO2t'], ['EX_glc__D_e'], ['EX_nh4_e'], ['

/home/touliopoulos/.local/lib/python3.10/site-packages/dingo/utils.py:381: ClusterWarning:

scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix



# Remove False positive clusters classifications

In [7]:

# find which reactions are in 2 clusters like PGK PGK_rev

# find in which cluster their metabolites are participating


BIGG_COFACTORS = ['atp[c]', 'atp_c', 'adp_c', 'adp[c]',
                  'atp[c]', 'atp_c', 'adp_c', 'adp[c]',
                  'udp[c]', 'udp_c', 'ump[c]', 'ump_c',
                  'amp_c', 'amp[c]',
                  'gdp[c]', 'gdp_c', 'gtp[c]', 'gtp_c',
                  'accoa_c', 'accoa[c]', 'coa_c', 'coa[c]',  # acetyl-CoA
                  'q8[c]', 'q8_c', 'q8h2_c', 'q8h2[c]', 'mqn8_c', 'mqn8[c]', 'mql8_c', 'mql8[c]', 'q8h2_c', 'q8h2[c]',
                  'actp[c]', 'actp_c',
                  'h2o_c', 'h2o[c]', 'h2o_e', 'h2o[e]',
                  'pi_e', 'pi[e]', 'pi_c', 'pi[c]', 'ppi[c]', 'ppi_c',
                  'pep_c', 'pep[c]',
                  'h_c', 'h[c]', 'h_e', 'h[e]',
                  'o2_c', 'o2[c]', 'o2_e', 'o2[e]',
                  'co2_c', 'co2[c]', 'co2_e', 'co2[e]',
                  'nadp_c', 'nadp[c]', 'nadph_c', 'nadph[c]', 'nad_c', 'nad[c]', 'nadh_c', 'nadh[c]',
                  'nadp_e', 'nadp[e]', 'nadph_e', 'nadph[c]', 'nad_e', 'nad[e]', 'nadh_e', 'nadh[e]',
                  'fadh2_c', 'fadh2[c]', 'fad_c', 'fad[c]',
                  'nh4_c', 'nh4[c]', 'nh4_e', 'nh4[e]',
                  'pyr[c]', 'pyr_c'
                ]

BIGG_BUILDING_BLOCKS = ['ala_L[c]', 'asp_L[c]', ' gln_L[c]', 'glu_L[c]', 'glu_L[c]', 'ser_L[c]', 'trp_L[c]', 'met_L[c]', 'lys_L[c]', 'cyst_L[c]']


def find_reactants_products(cobra_model, extended_reactions):
    
    # remove suffix in order to apply cobra functionalities to the id
    reactions_ids_removed_suffix = [item.removesuffix("_rev") for item in extended_reactions]

    reactants_list_all_reactions = []
    products_list_all_reactions = []
    last_parsed_reaction = ""

    for reaction in reactions_ids_removed_suffix:
        reactants_list_single_reaction = []
        products_list_single_reaction = []

        reaction_information = cobra_model.reactions.get_by_id(reaction)
        
        # if you have not already parsed the same reaction id (you are currently in the reverse)
        # and it is a case with both forward and reverse (preffix appears twice, avoid unique reactions with only forward case)
        if reaction != last_parsed_reaction and reactions_ids_removed_suffix.count(reaction) == 2:
            
            #reactants_list_single_reaction = []
            #products_list_single_reaction = []

            #reaction_information = cobra_model.reactions.get_by_id(reaction)

            products = reaction_information.reactants
            reactants = reaction_information.products
            
            for reactant in reactants:
                reactant = str(reactant)
                
                if reactant in BIGG_COFACTORS or reactant in BIGG_BUILDING_BLOCKS:
                    pass
                else:
                    reactants_list_single_reaction.append(reactant) 


            for product in products:
                product = str(product)
                if product in BIGG_COFACTORS or product in BIGG_BUILDING_BLOCKS:
                    pass
                else:
                    products_list_single_reaction.append(product)         

            reactants_list_all_reactions.append(reactants_list_single_reaction)
            products_list_all_reactions.append(products_list_single_reaction)


        # if you have already parsed the reverse reaction and you are in the forward case
        else:
            #reactants_list_single_reaction = []
            #products_list_single_reaction = []

            #reaction_information = cobra_model.reactions.get_by_id(reaction)

            # reverse
            reactants = reaction_information.reactants
            products = reaction_information.products
            
            for reactant in reactants:
                reactant = str(reactant)
                
                if reactant in BIGG_COFACTORS or reactant in BIGG_BUILDING_BLOCKS:
                    pass
                else:
                    reactants_list_single_reaction.append(reactant) 


            for product in products:
                product = str(product)
                if product in BIGG_COFACTORS or product in BIGG_BUILDING_BLOCKS:
                    pass
                else:
                    products_list_single_reaction.append(product)         

            reactants_list_all_reactions.append(reactants_list_single_reaction)
            products_list_all_reactions.append(products_list_single_reaction)
            
        # use this to handle rev reactions
        last_parsed_reaction = reaction
        
    reactions_dictionary = {a: {"reactants": b, "products": c} for a, b, c in zip(extended_reactions, reactants_list_all_reactions, products_list_all_reactions)}
        
    return reactions_dictionary


reactions_dictionary = find_reactants_products(ec_cobra_model, extended_reactions)
# what do with reactions that have no main reactants or products (only cofactors)
#print(reactions_dictionary)



#print(clusters_ward)
#print(labels_ward)
#print(extended_reactions)

def match_forward_reverse_reactions_to_clusters(clusters_ward):
    clusters_ward_copy = copy.deepcopy(clusters_ward)
    clusters_of_each_rev_reaction = []
    for cluster in clusters_ward_copy:
        for reaction in cluster:
            if reaction.endswith("_rev"):
                original_id = reaction.split("_rev")[0]
                #clusters_of_rev_reaction = [reaction, original_id]
                clusters_of_rev_reaction = [original_id]

                for cluster in clusters_ward_copy:
                    # when they are in the same cluster its appended only one time
                    if reaction in cluster or original_id in cluster:
                        # remove reaction from its cluster (avoid duplicates later)
                        try:
                            cluster.remove(original_id)
                        except:
                            pass
                        
                        try:
                            cluster.remove(reaction)
                        except:
                            pass
                        
                        clusters_of_rev_reaction.extend([cluster])
                        
                clusters_of_each_rev_reaction.append(clusters_of_rev_reaction)
            
    return clusters_of_each_rev_reaction
                

clusters_of_each_rev_reaction = match_forward_reverse_reactions_to_clusters(clusters_ward)
#print(clusters_of_each_rev_reaction)



def exclude_forward_or_reverse_reaction(reactions_dictionary, clusters_of_each_rev_reaction):
    reactions_to_remove = []
    
    for groups in clusters_of_each_rev_reaction:
        print(len(groups), groups)
        forward = groups[0]
        reverse = forward + "_rev"
        
        # only one cluster found, reactions share the same cluster
        if len(groups) == 3:
            cluster_1 = groups[1]
            cluster_2 = groups[2]
            
        # 2 clusters found, reactions are in different clusters
        # we want to find which reaction suits better to its cluster
            
            reactants_forward = reactions_dictionary[forward]["reactants"]
            products_forward = reactions_dictionary[forward]["products"]
            
            reactants_cluster_1 = [reactions_dictionary[reaction]["reactants"] for reaction in cluster_1 if reaction in reactions_dictionary]
            products_cluster_1 = [reactions_dictionary[reaction]["products"] for reaction in cluster_1 if reaction in reactions_dictionary]

            reactants_cluster_2 = [reactions_dictionary[reaction]["reactants"] for reaction in cluster_2 if reaction in reactions_dictionary]
            products_cluster_2 = [reactions_dictionary[reaction]["products"] for reaction in cluster_2 if reaction in reactions_dictionary]
            
            # double check in case of other reverse reactions present in the same cluster
            reactants_products_forward = reactants_forward + products_forward                
            
            # combine reactants and products of each cluster
            reactants_products_cluster_1 = reactants_cluster_1 + products_cluster_1
            reactants_products_cluster_2 = reactants_cluster_2 + products_cluster_2
            # flatten nested list
            reactants_products_cluster_1 = [item for sublist in reactants_products_cluster_1 for item in sublist]
            reactants_products_cluster_2 = [item for sublist in reactants_products_cluster_2 for item in sublist]
            
            #print(reactants_products_forward)
            #print(reactants_products_cluster_1)
            #print(reactants_products_cluster_2)
        
            count_forward_cluster_1 = len(set(reactants_products_forward) & set(reactants_products_cluster_1))
            count_forward_cluster_2 = len(set(reactants_products_forward) & set(reactants_products_cluster_2))
            
            #if (count_forward_cluster_1) > (count_forward_cluster_2):
            if (count_forward_cluster_1 > 0) and (count_forward_cluster_2 == 0):
                print("Keep", forward, cluster_1)
                print("Remove", forward, cluster_2)
                
                #print("Remove", reverse, cluster_1)
                #print("Keep", reverse, cluster_2)
                
                #if forward in cluster_2:
                #    reactions_to_remove.append(forward)
                #    cluster_2.remove(forward)
                #elif reverse in cluster_2:
                #    reactions_to_remove.append(reverse)
                #    cluster_2.remove(reverse)
                                    
            #elif (count_forward_cluster_1) < (count_forward_cluster_2):
            elif (count_forward_cluster_1 == 0) and (count_forward_cluster_2 > 0):
                print("Keep", forward, cluster_2)
                print("Remove", forward, cluster_1)
                
                #print("Remove", reverse, cluster_2)
                #print("Keep", reverse, cluster_1)
                
                #if forward in cluster_1:    
                #    cluster_1.remove(forward)
                #    reactions_to_remove.append(forward)
                #elif reverse in cluster_1:
                #    cluster_1.remove(reverse)
                #    reactions_to_remove.append(reverse)
                                    
            else:
                # keep random reaction 
                print("Random Cluster", forward)
                # or find biggesr connectivity
                
    print(reactions_to_remove)
            
            
            # keep indices or ids and remove afterward (after loop) cause we will lose information


exclude_forward_or_reverse_reaction(reactions_dictionary, clusters_of_each_rev_reaction)




3 ['RPI', ['PGL', 'PGM', 'RPE', 'TALA', 'TKT1', 'TKT2', 'G6PDH2r', 'GND'], ['PGI', 'PGM_rev', 'ENO', 'TPI', 'FBA', 'GAPD']]
Keep RPI ['PGL', 'PGM', 'RPE', 'TALA', 'TKT1', 'TKT2', 'G6PDH2r', 'GND']
Remove RPI ['PGI', 'PGM_rev', 'ENO', 'TPI', 'FBA', 'GAPD']
3 ['PGK', ['PGL', 'PGM', 'RPE', 'TALA', 'TKT1', 'TKT2', 'G6PDH2r', 'GND'], ['PGI', 'PGM_rev', 'ENO', 'TPI', 'FBA', 'GAPD']]
Random Cluster PGK
3 ['ACKr', ['ACt2r_rev', 'PTAr', 'EX_ac_e'], ['ACt2r']]
Random Cluster ACKr
3 ['D_LACt2', ['EX_lac__D_e'], []]
Keep D_LACt2 ['EX_lac__D_e']
Remove D_LACt2 []
3 ['LDH_D', ['EX_lac__D_e'], []]
Random Cluster LDH_D
3 ['ALCD2x', ['ETOHt2r_rev', 'EX_etoh_e'], ['ETOHt2r']]
Random Cluster ALCD2x
3 ['EX_o2_e', ['CYTBD', 'O2t'], []]
Random Cluster EX_o2_e
3 ['EX_pi_e', ['PIt2r', 'BIOMASS_Ecoli_core_w_GAM'], []]
Random Cluster EX_pi_e
3 ['SUCOAS', ['AKGDH', 'ICDHyr'], []]
Keep SUCOAS ['AKGDH', 'ICDHyr']
Remove SUCOAS []
3 ['ACALD', ['PFK', 'PPC', 'ATPM', 'PPCK', 'ATPS4r', 'PYK', 'THD2', 'EX_fru_e', 'EX_f

# Extract clusters

In [13]:
"""
cluster_of_interest_1 = clusters_complete[0]
cluster_of_interest_2 = clusters_complete[1]
print(len(cluster_of_interest_1))
print(len(cluster_of_interest_2))

groups_of_interest = []

for group in roseburia_cobra_model.groups:
    group_count_1 = 0
    group_count_2 = 0
    for reaction in group.members:
        if str(reaction.id) in cluster_of_interest_1:
            group_count_1 += 1
        elif str(reaction.id) in cluster_of_interest_2:
            group_count_2 += 1
            
    if group_count_1 != 0 and group_count_2 != 0:
        scaled_group_count_1 = group_count_1/len(cluster_of_interest_1)
        scaled_group_count_2 = group_count_2/len(cluster_of_interest_2)      
        if ( (scaled_group_count_1 / scaled_group_count_2) > 4):
            groups_of_interest.append(group)
            print(group, round(scaled_group_count_1*100, 2), round(scaled_group_count_2*100, 2))


print("\n")
"""


cluster_of_interest_1 = clusters_ward[0]
cluster_of_interest_2 = clusters_ward[1]
print(len(cluster_of_interest_1))
print(len(cluster_of_interest_2))

groups_of_interest = []


for group in roseburia_cobra_model.groups:

    group_count_1 = 0
    group_count_2 = 0

    for reaction in group.members:
        if str(reaction.id) in cluster_of_interest_1:
            group_count_1 += 1
        elif str(reaction.id) in cluster_of_interest_2:
            group_count_2 += 1
            
    if group_count_1 != 0 and group_count_2 != 0:
        print(group_count_1, group_count_2, len(group.members))
        scaled_group_count_1 = group_count_1*100/len(group.members)
        scaled_group_count_2 = group_count_2*100/len(group.members)
        print(group, round(scaled_group_count_1, 2), round(scaled_group_count_2, 2) )




10
8


NameError: name 'roseburia_cobra_model' is not defined

Get information on Acetate/Glucose participating reactions

In [41]:


#flux_variability_analysis(roseburia_cobra_model, roseburia_cobra_model.reactions[:10], fraction_of_optimum=0.9, loopless=False)


reaction_ids = ["EX_ac(e)", "ACtr" , 
                "EX_lac_D(e)", "D_LACt2" , 
                "EX_lac_L(e)" , "L_LACt2r"]

#reaction_ids = ["EX_glc_D(e)", "GLCabc" , 
#                "EX_fru(e)" , "FRUt2r" , 
#                "EX_gal(e)" , "GALt4" , "GALabc" , "GALt1r",
#                "EX_sucr(e)" , "SUCRt2" , 
#                "EX_malt(e)" , "MALTabc"]


# Filter reactions from the model based on the reaction IDs
reactions_of_interest = [roseburia_cobra_model.reactions.get_by_id(rxn_id) for rxn_id in reaction_ids]

# Perform FVA on the selected reactions
fva_results = flux_variability_analysis(
    roseburia_cobra_model, 
    reactions_of_interest, 
    fraction_of_optimum=0.99, 
    loopless=False
)

print(fva_results)


# Run FBA again with the new constraint
solution = roseburia_cobra_model.optimize()

reaction_ids = ["EX_ac(e)", "ACtr" , 
                "EX_lac_D(e)", "D_LACt2" , 
                "EX_lac_L(e)" , "L_LACt2r"]

reaction_ids = ["EX_glc_D(e)", "GLCabc" , 
                "EX_fru(e)" , "FRUt2r" , 
                "EX_gal(e)" , "GALt4" , "GALabc" , "GALt1r",
                "EX_sucr(e)" , "SUCRt2" , 
                "EX_malt(e)" , "MALTabc"]

# Extract the flux values for the selected reactions
flux_values = {rxn_id: solution.fluxes.get(rxn_id, None) for rxn_id in reaction_ids}

# Print the flux values
print("Flux values for selected reactions:")
for rxn_id, flux in flux_values.items():
    print(f"{rxn_id}: {flux}")



                 minimum      maximum
EX_ac(e)      -35.679864   220.986803
ACtr         -220.986803    35.679864
EX_lac_D(e) -1000.000000  1000.000000
D_LACt2     -1000.000000  1000.000000
EX_lac_L(e) -1000.000000  1000.000000
L_LACt2r    -1000.000000  1000.000000
Flux values for selected reactions:
EX_glc_D(e): 0.0
GLCabc: 0.0
EX_fru(e): 1000.0
FRUt2r: -1000.0
EX_gal(e): 0.0
GALt4: -376.67284662106925
GALabc: 0.0
GALt1r: 376.67284662106925
EX_sucr(e): -956.0599779732079
SUCRt2: 956.0599779732079
EX_malt(e): 0.0
MALTabc: 0.0


In [43]:
from scipy.stats import pearsonr


list1 = [-4, -2, 1, 3]
list2 = [-4, -3, -2, -1]
list3 = [-1, -2, -3, -4]

# Calculate Pearson correlation
corr_coefficient, p_value = pearsonr(list1, list2)
print(corr_coefficient)

corr_coefficient, p_value = pearsonr(list1, list3)
print(corr_coefficient)

corr_coefficient, p_value = pearsonr(list2, list3)
print(corr_coefficient)



0.9965457582448798
-0.9965457582448798
-1.0
0
